In [1]:
import json
from typing import Annotated

import nest_asyncio
import requests
import uvicorn
from fastapi import FastAPI, Query,Path,Body
from pydantic import BaseModel

In [5]:
class Item(BaseModel):
    name: str
    description: str | None = None
    price: float
    tax: float | None = None


class User(BaseModel):
    username: str
    full_name: str | None = None

In [28]:
importance = 1
#isinstance(Query())
type(Query()).__name__

'Query'

In [13]:
app = FastAPI()


@app.get("/index/")
async def index(data: Annotated[str | None, Query(alias="query-data")]):
    return data


if __name__ == "__main__":
    nest_asyncio.apply()
    uvicorn.run(app, port=4444)

INFO:     Started server process [94715]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:4444 (Press CTRL+C to quit)


INFO:     127.0.0.1:38132 - "GET /index/?query-data=123 HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [94715]
